In [ ]:
!pip install openai
!pip install nltk
!pip install pytesseract
!pip install pdf2image
!pip install pillow

In [16]:
import random
import json
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import openai
import string
import re
import pytesseract
import pdf2image
from PIL import Image
import os
from pdf2image import convert_from_path

Defines the Extractive Summary function, which identifies sentences with frequently occuring words, as well as a list of keywords. 

In [2]:
def Extractive_Summary(corpus, keywords = [], max_chars = 2500):
    print("starting extractive summary")
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(corpus)
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1
    sentences = corpus.replace("!", ".").replace("?", ".").split(".")
    print("Generated " + str(len(sentences)) + " sentences")
    sentenceValue = dict()
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]
    # Average value of a sentence from the original text
    average = int(sumValues / len(sentenceValue))
    print("Average: " + str(average))
    print("Generating summary...")
    # Storing sentences into our summary.
    summary = []
    ct = 0
    for sentence in sentences:
        if (sentence in sentenceValue):
            summary.append({"sentence": sentence, "value": sentenceValue[sentence], "index": ct})
            ct += 1
    ct = 0
    summary.sort(key=lambda x: x["value"], reverse=False)
    summary_str = ""
    i=0
    print("Part 2 of summary generation...")
    summary_sentence_list = []
    for i in summary:
        if any(ele in i["sentence"] for ele in keywords):
            #print("keyword found")
            ct += 1
            summary_sentence_list.append(i)
    for i in summary:
        #print("ssl: ", json.dumps(summary_sentence_list))
        if (i not in summary_sentence_list) and (sum(len(s["sentence"]) for s in summary_sentence_list) < max_chars):
            #print("Sentence added")
            ct += 1
            summary_sentence_list.append(i)
    summary_sentence_list.sort(key=lambda x: x["index"], reverse=False)
    for i in summary_sentence_list:
        summary_str += i["sentence"] + "/n"
    #print("Percent reduction: " + str(len(summary_str)/len(data)) + "%")
    print("Sentences in summary: " + str(ct))
    return summary_str
    

Installs dependenceis

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\joeya\AppData\Local\Tesseract-OCR\tesseract.exe'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joeya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joeya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Creates corpus.txt from a pdf. Only use this if you are not manually creating corpus.txt and wish to create it from a PDF

In [3]:
def PDF_OCR(filename):
    pages = convert_from_path(os.getcwd() +"/files/" + filename, 500)
    print("Converting PDF to images...")
    for i in range(len(pages)):
        pages[i].save(os.getcwd() + '/images/page'+str(i)+'.jpg', 'JPEG')
    print("Converting images to text...")
    text = ""
    for i in range(len(pages)):
        text += pytesseract.image_to_string(Image.open(os.getcwd() + '/images/page'+str(i)+'.jpg'))
    print("Cleaning up...")
    for i in range(len(pages)):
        os.remove(os.getcwd() + '/images/page'+str(i)+'.jpg')
    return text

#write text to file
with open("corpus.txt", "w") as f:
    f.write(PDF_OCR(input("enter filename: ")))
f.close()

Converting PDF to images...
Converting images to text...
Cleaning up...


Reads the corpus

In [17]:
text_file = open("corpus.txt", "r")
data = text_file.read()
text_file.close()


Summary generation based on user input

In [10]:
input_length = input("How long do you want the summary to be (in characters)? The corpus is " + str(len(data)) + " characters long. ")
print("estimated " + str(int(input_length)/len(data)*100) + "% reduction")
print("That input will cost an estimated $" + str(0.02*int(input_length)/4000))
output_length = int(input("How long do you want your answers to br at maximum? (in characters), if you do not want to limit, type n, otherwise enter a number smaller than " + str(15000 - int(input_length)))) 
print(input_length, output_length)
print("total cost: $" + str((output_length*0.02)/4000 + (int(input_length)*0.02)/4000))
keywords = []
word = ""
print("Press q to stop adding keywords")
while keywords != "q":
    word = input("Enter a keyword: ")
    if word == "q":
        break
    keywords.append(word)
print("Generating summary...")
summary = Extractive_Summary(data, max_chars = int(input_length))


estimated 20.76196408180214% reduction
That input will cost an estimated $0.05
10000 1500
total cost: $0.0575
Press q to stop adding keywords
Generating summary...
starting extractive summary
Generated 391 sentences
Average: 968
Generating summary...
Part 2 of summary generation...
Sentences in summary: 110


In [18]:
openai.api_key = #Your API key here

Prompts user for questions, and retrieves response from OpenAI api

In [20]:
question = input("Enter your question: ")
prompt = summary + "\n\n\n Based on this passage, " + question
temp = int(input("On a scale of 1 to 10, how creative would you like your response to be"))/10

response = openai.Completion.create(
  model="text-davinci-002",
  prompt=prompt,
  temperature=temp,
  max_tokens=int(output_length/4),
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(question)
print(response["choices"][0]["text"])
print("total cost: $" + str((response["usage"]["total_tokens"]/1000)*0.02))


how does moksha connect to this text?


Moksha is mentioned in the Bhagavad Gita in connection with the idea of renunciation. Krishna tells Arjuna that renunciation is difficult to attain without discipline, and that a sage armed with discipline soon reaches the infinite spirit. He also says that those who find fault and fail to follow his thought are lost fools, deluded by every bit of knowledge. Therefore, moksha could be seen as a state of clarity or enlightenment that is attained through discipline and renunciation.
total cost: $0.066
